(acoustics-receiver_hydrophone_sensitivity)=
## Hydrophone and receiving sensitivity 

(acoustics-receiver_hydrophone)=
### Hydrophone
The simplest instrument we can use to capture sound in the ocean is a hydrophone. Hydrophone is a "transducer" that can convert sound pressure into voltage, which can then be recorded into a computer or a recorder. 

```{Note}
:class: note
Transducers can also convert voltage into pressure and are used as acoustic sources (transmitters), which are discussed in the [Acoustic sources](acoustics-source) tutorial.
```

In [1]:
# HERE WE CAN BRING IN AN IN-DEPTH PAGE TALKING ABOUT PIEZOELECTRIC MATERIALS

Hydrophones come in all sizes and shapes, but the commonest ones all look like a blob connected with a cable, like those shown below:

ADD HYDROPHONE PICTURES

The "blob" usually is where the transudcer element is located, sometimes with supporting electronics, all encapsulated in polyurethane to protect these elements from the environment.

(acoustics-receiver_sensitivity)=
### Receiving sensitivity

When measuring sound underwater, an important consideration is the _receiving sensitivity_, which characterizes how well the hydrophone can convert sound at different frequencies into voltage signals. Therefore, usually the receiving sensitivty is expressed in values and units like **-180 dB re 1V/µPa**, which means that this hydrophone will produce a voltage of -180 dB re 1V, or $10^{-9}$ V (see [what decibel (dB) means here](primer-decibel)) in response to a sound pressure of 1 micropascal (µPa). The negative value here just means that the conversion factor is a small number. The receiving sensitivity is what we are after when trying to _calibrate_ a hydrophone.

In [ ]:
# ADD WIDGET:
# - ALLOW CHANGING SENSITIVTY AND INPUT VOLTAGE
# - GOAL IS TO DEVELOP THE INTUITION THAT INCREASING SENSITIVITY (MAKING IT LESS NEGATIVE) WOULD PRODUCE A LARGER VOLTAGE SIGNAL

In [2]:
import numpy as np
import ipywidgets as widgets

def convertSPLtoV(dBSPL_re_20uPa, sens_dB_M=-38, mic_gain_dB_at_f=12, preamp_gain=200, reference_pressure=20e-6):
    SPL_reference_level = 20*np.log10(1/reference_pressure)
    preamp_gain_dB = 20*np.log10(preamp_gain)
    system_gain_dB_G = mic_gain_dB_at_f +  preamp_gain_dB
    measured_voltage_dB = dBSPL_re_20uPa + system_gain_dB_G + sens_dB_M - SPL_reference_level
    return measured_voltage_dB

def display_voltage(SPL_dB, sens_dB_M, mic_gain_dB_at_f, preamp_gain):
    v_dB = convertSPLtoV(SPL_dB, sens_dB_M=sens_dB_M, mic_gain_dB_at_f=mic_gain_dB_at_f, preamp_gain=preamp_gain)
    v_linear = 10 ** (v_dB / 20)
    print(f"SPL: {SPL_dB} dB → Voltage: {v_linear:.6f} V")

widgets.interact(
    display_voltage,
    SPL_dB=widgets.FloatSlider(value=100, min=80, max=180, step=0.5, layout=widgets.Layout(width="1000px"), style={'description_width': 'initial'}, description="SPL (dB)"),
    sens_dB_M=widgets.FloatSlider(value=-36, min=-60, max=-20, step=0.5, layout=widgets.Layout(width="1000px"), style={'description_width': 'initial'}, description="Sensitivity (dBV/Pa)"),
    mic_gain_dB_at_f=widgets.FloatSlider(value=0, min=0, max=40, step=1, layout=widgets.Layout(width="1000px"), style={'description_width': 'initial'}, description="Mic Gain (dB)"),
    preamp_gain=widgets.FloatSlider(value=1, min=1, max=1000, step=1, layout=widgets.Layout(width="1000px"), style={'description_width': 'initial'}, description="Preamp Gain")
)

interactive(children=(FloatSlider(value=100.0, description='SPL (dB)', layout=Layout(width='1000px'), max=180.…

<function __main__.display_voltage(SPL_dB, sens_dB_M, mic_gain_dB_at_f, preamp_gain)>

(acoustics-receiver_metrics)=
## Quantifying received sound

Once sound signals are captured by hydrophones, how do we quantify what's in the received sound?

### Sound pressure level (SPL)
One of the commonest measure is **sound pressure level (SPL)**, which characterizes how loud the sound is and is usually written as:

$\textrm{SPL} = 20 \log{ \frac{P}{P_{ref}} }$

where $P$ is the sound pressure, and $P_{ref}$ is the reference pressure. This is why the unit of SPL is typically in the form of **dB re 1 µPa**, as it is a logarithmic measure of the ratio between the sound pressure and a reference pressure.

```{Note}
:class: note
The reference pressure is usually 1 µPa in water and 20 µPa in air, which makes a big difference in the resulting SPL values!
```

In practice, to calculate SPL from hydrophone measurement, we often use:

$\textrm{SPL} = 20 \log{ \frac{V}{V_{ref}} } - M$,

where $V$ is the measured voltage and $M$ is the receiving sensitivity. To see how this formula comes about, let's start with the linear domain quantities:

$V = P \times m$,

where $m$ is the linear receiving sensitivty (i.e. $M=20 \log{m}$). Dividing both sides by $V_{ref}$ and multiply the right hand side by $\frac{P_{ref}}{P_{ref}}$, we get

$\frac{V}{V_{ref}} = \frac{P}{P_{ref}} \times \frac{P_{ref}}{V_{ref}} \times m$.

Taking the equation to the log domain:

$20 \log{ \frac{V}{V_{ref}} } = 20 \log{ \frac{P}{P_{ref}} } + 20 \log{ \frac{m}{ V_{ref} / P_{ref}} }$,

which gives

$20 \log{ \frac{V}{V_{ref}} } = \textrm{SPL} + M$,

and hence the above formula.

When there is a gain factor $G$ applied to the voltage signal by electronics associated with the hydrophone, we have

$\textrm{SPL} = 20 \log{ \frac{V}{V_{ref}} } - M - G$.

Here there is a minus sign before $G$, since to return to the original measured quantity, we need _remove_ the influence of the gain.


### Sound pressure metrics

However, most sound recordings consist of a series of voltage measurements. How do we choose a point to calculate the SPL, and is that even the right way to do it?

Indeed usually we do not report a single point on a voltage time series, but use a metric or a descriptor. Most metrics are based either directly on sound pressure, or on sound intensity $I$, which is proportional to the square of sound pressure ($I = \frac{P^2}{\rho c}$). Precisely because the goal is to quantify sound received in a time series, you will notice the time period under consideration ($T$) in all of the common metrics below:

- Root mean square (RMS) pressure [unit: Pa]: 

  $P_\textrm{RMS} = \sqrt{ \frac{\int_T P(t)^2 dt}{T} }$

- Peak-to-peak (p2p) pressure [unit: Pa]:

  $P_\textrm{p2p} = P_\textrm{max} - P_\textrm{min}$ during the time period under consideration

- Energy flux density (EFD) [unit: W/m$^2$]:
  
  $E = \int_T I(t) dt$

Different from RMS and p2p pressure, EFD is a cumulative measure and is often used to quantify sound energy exposure for marine animals.



```{Tip}
:class: tip
You may already know each signal has a specific frequency content, or "spectrum." The signal spectrum is very important in distinguishing different types of signals and many other applications. Learn more in the [Spectral analysis](acoustics-spectral) tutorial.
```

In [ ]:
# ADD WIDGET:
# - SIGNAL TYPES: IMPULSE, TAPERED SINE WAVE (NO TAPER TO FULLY TAPERED)
# - ALLOW CHANGING SINE WAVE LENGTH, AMPLITUDE, TAPERING
# - ALLOW CHANGING IMPULSE AMPLITUDE
# - PLOT RMS, P2P LEVEL ON THE FIGURE
# - SHOW EFD, RMS, P2P NUMBERS AS OUTPUTS


# NOTE: LEAVING ANYTHING IN THE FREQ DOMAIN TO SPECTRAL ANALYSIS TUTORIAL
